In [3]:
import os 

# get the current working directory 
%pwd

# change the working directory to parent directory
working_dir = "G:/ML_DL_Projects/MLOPS_Indian_Flight_Price_Prediction"
os.chdir(working_dir)

print(f"Current working directory is {os.getcwd()} ")

Current working directory is G:\ML_DL_Projects\MLOPS_Indian_Flight_Price_Prediction 


In [11]:
import pandas as pd

# economy data 
data_economy = pd.read_csv("artifacts/data_ingestion/economy.csv")
data_economy.head(5)

,date,airline,ch_code,num_code,dep_time,from,time_taken,stop,arr_time,to,price
0,11-02-2022,SpiceJet,SG,8709,18:55,Delhi,02h 10m,non-stop,21:05,Mumbai,"5,953"
1,11-02-2022,SpiceJet,SG,8157,06:20,Delhi,02h 20m,non-stop,08:40,Mumbai,"5,953"
2,11-02-2022,AirAsia,I5,764,04:25,Delhi,02h 10m,non-stop,06:35,Mumbai,"5,956"
3,11-02-2022,Vistara,UK,995,10:20,Delhi,02h 15m,non-stop,12:35,Mumbai,"5,955"
4,11-02-2022,Vistara,UK,963,08:50,Delhi,02h 20m,non-stop,11:10,Mumbai,"5,955"


In [13]:
# business data
data_business = pd.read_csv("artifacts/data_ingestion/business.csv")
data_business.head(5)

,date,airline,ch_code,num_code,dep_time,from,time_taken,stop,arr_time,to,price
0,11-02-2022,Air India,AI,868,18:00,Delhi,02h 00m,non-stop,20:00,Mumbai,"25,612"
1,11-02-2022,Air India,AI,624,19:00,Delhi,02h 15m,non-stop,21:15,Mumbai,"25,612"
2,11-02-2022,Air India,AI,531,20:00,Delhi,24h 45m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,20:45,Mumbai,"42,220"
3,11-02-2022,Air India,AI,839,21:25,Delhi,26h 30m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,23:55,Mumbai,"44,450"
4,11-02-2022,Air India,AI,544,17:15,Delhi,06h 40m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,23:55,Mumbai,"46,690"


In [14]:
# columns in economy.csv data
data_economy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206774 entries, 0 to 206773
Data columns (total 11 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   date        206774 non-null  object
 1   airline     206774 non-null  object
 2   ch_code     206774 non-null  object
 3   num_code    206774 non-null  int64 
 4   dep_time    206774 non-null  object
 5   from        206774 non-null  object
 6   time_taken  206774 non-null  object
 7   stop        206774 non-null  object
 8   arr_time    206774 non-null  object
 9   to          206774 non-null  object
 10  price       206774 non-null  object
dtypes: int64(1), object(10)
memory usage: 17.4+ MB


In [15]:
data_business.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93487 entries, 0 to 93486
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   date        93487 non-null  object
 1   airline     93487 non-null  object
 2   ch_code     93487 non-null  object
 3   num_code    93487 non-null  int64 
 4   dep_time    93487 non-null  object
 5   from        93487 non-null  object
 6   time_taken  93487 non-null  object
 7   stop        93487 non-null  object
 8   arr_time    93487 non-null  object
 9   to          93487 non-null  object
 10  price       93487 non-null  object
dtypes: int64(1), object(10)
memory usage: 7.8+ MB


In [17]:
print(data_economy.isnull().sum())
print(data_business.isnull().sum())

date          0
airline       0
ch_code       0
num_code      0
dep_time      0
from          0
time_taken    0
stop          0
arr_time      0
to            0
price         0
dtype: int64
date          0
airline       0
ch_code       0
num_code      0
dep_time      0
from          0
time_taken    0
stop          0
arr_time      0
to            0
price         0
dtype: int64


In [18]:
print(data_business.shape)
print(data_economy.shape)

(93487, 11)
(206774, 11)


In [49]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataValidationConfig:
    # from config.yaml
    root_dir : Path
    # unzip_data_dir : Path
    unzip_data_files : list[Path]
    STATUS_FILE : str
    # from schema.yaml
    all_schema : dict

In [50]:
from src.indian_flight_price_prediction.constants import *
from src.indian_flight_price_prediction.utils.common import read_yaml, create_directories

In [56]:
# for src/<project_name>config/configuration.py
class ConfigurationManager:
    def __init__(
            self,
            config_file_path = CONFIG_FILE_PATH,
            params_file_path = PARAMS_FILE_PATH,
            schema_file_path = SCHEMA_FILE_PATH):
        
            self.config = read_yaml(config_file_path)
            self.params = read_yaml(params_file_path)
            self.schema = read_yaml(schema_file_path)

            create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:
          config = self.config.data_validation
          schema = self.schema.COLUMNS

          create_directories([config.root_dir])

          data_validation_config = DataValidationConfig(
                root_dir = config.root_dir,
                unzip_data_files = [Path(f) for f in config.unzip_data_files],
            #     unzip_data_dir = config.unzip_data_dir,
                STATUS_FILE = config.STATUS_FILE,
                all_schema = schema,  
          )

          return data_validation_config

In [57]:
import os
from src.indian_flight_price_prediction import logger


In [68]:
class DataValidation:
    def __init__(self, config : DataValidationConfig):
        self.config = config


    def validate_all_columns(self) ->bool:
        try:
            validation_status = None
            
            # data = pd.read_csv(self.config.unzip_data_dir)
            # all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            with open(self.config.STATUS_FILE, 'a') as f:
                for file_path in self.config.unzip_data_files:
                    data = pd.read_csv(file_path)
                    all_cols = list(data.columns)
                    f.write(f"\n{'='*27}\n")
                    f.write(f"Validation Report: {file_path.name}\n")
                    f.write(f"{'='*27}\n")
                    for col in all_cols:
                        if col not in all_schema:
                            validation_status = False
                            f.write(f"Column {col} not found in schema.Validation Status : {validation_status}.\n")
                        else:
                            if data[col].dtype == self.config.all_schema[col]:
                                validation_status = True
                                expected_dtype = str(self.config.all_schema[col])
                                actual_dtype = str(data[col].dtype)
                                f.write(f"✔ Column: {col:<13} | Expected: {expected_dtype:<7} | Found: {actual_dtype:<7} | Match \n")
                            else:
                                validation_status = False
                                f.write(f"✔ Column: {col:<13} | Expected: {expected_dtype:<7} | Found: {actual_dtype:<7} | Mis-match \n")

                return validation_status
        
        except Exception as e:
            raise e

In [69]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config = data_validation_config)
    data_validation.validate_all_columns()
    
except Exception as e:
    raise e

[2025-04-21 21:16:19,120 : INFO : common : yaml file : config\config.yaml loaded successfully.]
[2025-04-21 21:16:19,123 : INFO : common : yaml file : params.yaml loaded successfully.]
[2025-04-21 21:16:19,128 : INFO : common : yaml file : schema.yaml loaded successfully.]
[2025-04-21 21:16:19,130 : INFO : common : Created directory at artifacts]
[2025-04-21 21:16:19,132 : INFO : common : Created directory at artifacts/data_validation]
